In [ ]:
# Problem: The raw walkability data is ~1.4 GB. The Neon Postgres DB free tier only allows for ~500 MB. 
# The geometry is what is taking up most of the space. The file without the geometry is ~40 MB
# Challenge: Figure out how to reduce the size of the data to fit into the DB.

## The CRS needs to be 4326!!!

In [2]:
import geopandas as gpd
from shapely import wkt
import pandas as pd
from joblib import Parallel, delayed
import os

# Convert the .gdb to .csv

In [3]:
## This is working. Will probably need to move this elsewhere
file_path = r'data\WalkabilityIndex\Natl_WI.gdb' # This data is in NAD83 (EPSG:4269)

print("Reading the file from:", file_path)
gdf = gpd.read_file(file_path)
print("File read successfully.")

# Convert all column names to lowercase
gdf.columns = [col.lower() for col in gdf.columns]

# Detect and print the CRS
if gdf.crs:
    print("The CRS of the GeoDataFrame is:", gdf.crs)
else:
    print("The GeoDataFrame does not have a CRS.")

# Converting to EPSG:4326 because this is what we need for geographic coordinates
if gdf.crs and gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)
    print("CRS transformed to EPSG:4326.")
else:
    gdf.set_crs(epsg=4326, allow_override=True)
    print("CRS set to EPSG:4326.")

# Convert geometries to WKT format
gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)

csv_path_with_geometry = r'data\WalkabilityIndex\Natl_WI.csv'
print("Saving GeoDataFrame to CSV with geometry in WKT format at:", csv_path_with_geometry)
gdf.to_csv(csv_path_with_geometry, index=False)
print("GeoDataFrame saved to CSV with geometry in WKT format.")

gdf.head(n=1)

Reading the file from: data\WalkabilityIndex\Natl_WI.gdb
File read successfully.
The CRS of the GeoDataFrame is: PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102039"]]
CRS transformed to EPSG:4326.


C:\Users\grant\AppData\Local\Temp\ipykernel_9000\2408063383.py:26: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)


Saving GeoDataFrame to CSV with geometry in WKT format at: data\WalkabilityIndex\Natl_WI.csv
GeoDataFrame saved to CSV with geometry in WKT format.


,geoid10,geoid20,statefp,countyfp,tractce,blkgrpce,csa,csa_name,cbsa,cbsa_name,...,d3b,d4a,d2a_ranked,d2b_ranked,d3b_ranked,d4a_ranked,natwalkind,shape_length,shape_area,geometry
0,481130078254,481130078254,48,113,007825,4,206,"Dallas-Fort Worth, TX-OK",19100,"Dallas-Fort Worth-Arlington, TX",...,115.981747,362.1,6.0,14.0,15.0,17.0,14.0,3110.36082,297836.08309,MULTIPOLYGON (((-96.74178100024956 32.89033799...


# Simplify Geometry

In [20]:
## This works
# Load the walkability data using pandas
df = pd.read_csv(r'data\WalkabilityIndex\Natl_WI.csv')

# Convert the DataFrame to a GeoDataFrame and set the CRS to EPSG:4326
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'].apply(wkt.loads), crs='EPSG:4326')

print(f"Memory usage before simplification: {gdf.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

# Function to simplify geometry
def simplify_geometry(geom):
    return geom.simplify(0.0002, preserve_topology=True)

# Simplify the geometries using parallel processing
gdf['geometry'] = Parallel(n_jobs=-2)(delayed(simplify_geometry)(geom) for geom in gdf['geometry'])

print(f"Memory usage after simplification: {gdf.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

# Convert geometries to WKT and save to CSV
gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)
gdf.to_csv(r'data\WalkabilityIndex\Natl_WI_simplified.csv', index=False)

# Print file sizes
original_file_size = os.path.getsize(r'data\WalkabilityIndex\Natl_WI.csv') / 1024 ** 2
simplified_file_size = os.path.getsize(r'data\WalkabilityIndex\Natl_WI_simplified.csv') / 1024 ** 2

print(f"Original file size: {original_file_size:.2f} MB")
print(f"Simplified file size: {simplified_file_size:.2f} MB")

Memory usage before simplification: 76.50 MB
Memory usage after simplification: 76.50 MB


C:\Users\grant\AppData\Local\Temp\ipykernel_9000\2031908018.py:20: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)


Original file size: 2550.96 MB
Simplified file size: 386.80 MB


# Drop rows with no CSBA Name
Doing this to save memory and space. For v1, we're just interested in places that are part of a CBSA.

In [21]:
# Load the walkability data using pandas
input_path = r'data\WalkabilityIndex\Natl_WI_simplified.csv'
df = pd.read_csv(input_path)

# Drop rows where cbsa_name is null
df = df.dropna(subset=['cbsa_name'])

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'].apply(wkt.loads), crs='EPSG:4326')

# Print memory usage before saving
print(f"Memory usage before saving: {gdf.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

# Convert geometries to WKT and save to CSV
output_path = r'data\WalkabilityIndex\Natl_WI_simplified_drop_cols.csv'
gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)
gdf.to_csv(output_path, index=False)

# Print memory usage after saving
print(f"Memory usage after saving: {gdf.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

# Print file sizes
original_file_size = os.path.getsize(input_path) / 1024 ** 2
simplified_file_size = os.path.getsize(output_path) / 1024 ** 2

print(f"Original file size: {original_file_size:.2f} MB")
print(f"Simplified file size: {simplified_file_size:.2f} MB")

Memory usage before saving: 73.35 MB


C:\Users\grant\AppData\Local\Temp\ipykernel_9000\3833980465.py:16: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)


Memory usage after saving: 322.85 MB
Original file size: 386.80 MB
Simplified file size: 301.93 MB


# Optimize PG Column Data Types

In [22]:
# Ensure the geoid20 column is treated as strings
gdf.loc[:, 'geoid20'] = gdf['geoid20'].astype(str)

# Calculate the minimum and maximum length of the geoid20 column
min_length = gdf['geoid20'].str.len().min()
max_length = gdf['geoid20'].str.len().max()

print(f"Minimum length of geoid20: {min_length}")
print(f"Maximum length of geoid20: {max_length}")

Minimum length of geoid20: 11
Maximum length of geoid20: 12


C:\Users\grant\AppData\Local\Temp\ipykernel_9000\3980245470.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['481130078254' '481130078252' '481130078253' ... '720210312021'
 '720531504003' '720531501044']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf.loc[:, 'geoid20'] = gdf['geoid20'].astype(str)


In [23]:
min_value = gdf['natwalkind'].min()
max_value = gdf['natwalkind'].max()
print(f"Minimum natwalkind: {min_value}")
print(f"Maximum natwalkind: {max_value}")

Minimum natwalkind: 1.0
Maximum natwalkind: 20.0


In [24]:
# Given that we need to keep the database under ~500 MB, it might make sense to use the ranked values in the database
ranked_states = ["d2a_ranked","d2b_ranked", "d3b_ranked", "d4a_ranked"]

for column in ranked_states:
    min_value = gdf[column].min()
    max_value = gdf[column].max()
    print(f"Minimum {column}: {min_value:.2f}")
    print(f"Maximum {column}: {max_value:.2f}")


Minimum d2a_ranked: 1.00
Maximum d2a_ranked: 20.00
Minimum d2b_ranked: 1.00
Maximum d2b_ranked: 20.00
Minimum d3b_ranked: 1.00
Maximum d3b_ranked: 20.00
Minimum d4a_ranked: 1.00
Maximum d4a_ranked: 20.00


# Estimate PG Database Size

In [25]:
# This results in a pretty good estimate of the database. 
def calculate_data_sizes(df):
    # Specify the columns in the dataframe
    columns = ['geoid20', 'natwalkind', 'geometry']
    df = df.loc[:, columns]  # Use .loc to avoid SettingWithCopyWarning
    total_rows = len(df)
    
    # Convert the geometry column to Shapely objects
    df.loc[:, 'geometry'] = df['geometry'].apply(wkt.loads)
    
    # Calculate actual average size of each column
    geoid20_size_bytes = df['geoid20'].apply(lambda x: len(str(x).encode('utf-8'))).mean()
    geoid20_size_mb = geoid20_size_bytes / (1024 * 1024)
    
    natwalkind_size_bytes = df['natwalkind'].apply(lambda x: 4).mean()  # NUMERIC(4, 2) is 4 bytes
    natwalkind_size_mb = natwalkind_size_bytes / (1024 * 1024)
    
    # For geometry, we need to load it as a GeoDataFrame to calculate the actual size
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    # Set the initial CRS (assuming the initial CRS is EPSG:4326, change if different)
    gdf.set_crs(epsg=4326, inplace=True)
    
    # Calculate the average size of the geometries in bytes
    geometry_size_bytes = gdf['geometry'].apply(lambda x: len(x.wkb)).mean()
    geometry_size_mb = geometry_size_bytes / (1024 * 1024)
    
    # Calculate size per row
    size_per_row_mb = geoid20_size_mb + natwalkind_size_mb + geometry_size_mb
    
    # Calculate total data size (in MB)
    total_data_size_mb = size_per_row_mb * total_rows
    
    # Estimate index size (25% of data size)
    index_size_mb = total_data_size_mb * 0.25
    
    # Estimate overhead (10% of data size)
    overhead_mb = total_data_size_mb * 0.10
    
    # Total estimated database size
    total_db_size_mb = total_data_size_mb + index_size_mb + overhead_mb
    
    # Create a DataFrame with the results
    results = pd.DataFrame({
        'Metric': [
            'Total rows',
            'Average geoid20 size (MB)',
            'Average natwalkind size (MB)',
            'Average geometry size (MB)',
            'Estimated total data size (MB)',
            'Estimated total index size (MB)',
            'Estimated total overhead (MB)',
            'Estimated database size (MB)'
        ],
        'Value': [
            total_rows,
            geoid20_size_mb,
            natwalkind_size_mb,
            geometry_size_mb,
            total_data_size_mb,
            index_size_mb,
            overhead_mb,
            total_db_size_mb
        ]
    })
    
    return results

In [26]:
# Call the function
filepath_1 = r'data\WalkabilityIndex\Natl_WI_simplified.csv'
filepath_2 = r'data\WalkabilityIndex\Natl_WI_simplified_drop_cols.csv'

df1 = pd.read_csv(filepath_1)
results_df1 = calculate_data_sizes(df1)

df2 = pd.read_csv(filepath_2)
results_df2 = calculate_data_sizes(df2)

In [27]:
# Merge the results for comparison
comparison_df = results_df1.merge(results_df2, on='Metric', suffixes=('_file1', '_file2'))

# Add a column for percent change
comparison_df['Percent Change'] = round(((comparison_df['Value_file2'] - comparison_df['Value_file1']) / comparison_df['Value_file1']) * 100, 2)

# Rename the columns
comparison_df.columns = ['Metric', 'Simplified df', 'Simplified df, dropped cols', 'Percent Change']

# Save the comparison to a CSV file
comparison_df.to_csv(r'data\db_compression_results\geometry_simplification_and_column_drop.csv', index=False)
comparison_df

,Metric,Simplified df,"Simplified df, dropped cols",Percent Change
0,Total rows,220739.000000,203645.000000,-7.74
1,Average geoid20 size (MB),0.000011,0.000011,-0.06
2,Average natwalkind size (MB),0.000004,0.000004,0.00
3,Average geometry size (MB),0.000617,0.000503,-18.49
4,Estimated total data size (MB),139.577964,105.527562,-24.40
5,Estimated total index size (MB),34.894491,26.381891,-24.40
6,Estimated total overhead (MB),13.957796,10.552756,-24.40
7,Estimated database size (MB),188.430251,142.462209,-24.40


With a compression of .0002, the database in neon is 152 MB. The estimate was 142 MB